In [1]:
# Install following package:
#pip install squarify

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Show code')

In [3]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import pandas as pd
import numpy as np

###############################################

# Modules for widgets
import ipywidgets as wg
from IPython.display import display
import warnings
import squarify
warnings.simplefilter('ignore')

# Modules for delay
import time
from collections import Counter
from collections import defaultdict

In [4]:
df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = "ISO-8859-1", low_memory=False)
df_wgi = pd.read_csv('WorldBank_Data.csv', encoding = "ISO-8859-1", low_memory=False)
df1 = pd.read_csv('democracies.csv', encoding = "ISO-8859-1", low_memory=False)

df.fillna(0, inplace=True)
df_wgi.replace("..", 100, inplace=True)
df_wgi.dropna(inplace=True)

In [5]:
year = wg.IntSlider(
    value=2017,
    min=1996,
    max=2017,
    step=1,
    description='Year: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [6]:
def Scatterplot(year):
    dt = []
    cyear = str(year) + " [YR" + str(year) + ']'
    cyear1 = df_wgi.loc[df_wgi[cyear]]
    cyear2= df_wgi.loc[:, ['Country Name', 'Series Code',cyear]]
    tyear = df.loc[df['iyear'] == year]
    countries = tyear['country_txt'].unique()

    for country in countries:
        cyear3 = cyear2.loc[cyear2['Series Code'] == 'PV.EST']
        cyear4 = cyear3.loc[cyear3['Country Name']== country]
        Political_stability= float(cyear4[cyear].sum())
        cur_ct_terrorism = tyear[tyear['country_txt'] == country]
        aanslagen = 0
        for row in cur_ct_terrorism['country_txt']:
            aanslagen += 1
        if Political_stability >= -5 and Political_stability <= 5:    
            trace0 = go.Scatter(
                x = [Political_stability],
                y = [aanslagen],
                mode = 'markers',
                marker = dict(
                    size = 10,
                ),
                name = country,
                text = country
            )
            dt.insert(0, trace0)

    layout = go.Layout(
        autosize = True,
        title = go.layout.Title(
            text ="Aantal aanslagen tegenover Politieke Stabiliteit (" + str(year) + ")",
        ),
        xaxis=go.layout.XAxis(
            title = 'Political_stability',
            autorange = True,
        ),
        yaxis = go.layout.YAxis(
            type = 'log',
            title = 'Aanslagen',
            autorange = True,
        ),
    )
    if dt == []:
        print("No data available for this time period.")
    fig = go.Figure(data= dt, layout=layout)
    iplot(fig)

In [7]:
democracy = []
other = []
year_dif = 0

for i in range(len(df1['Entity'])):
    try:
        if year_dif < int(df1['Age of democracies at the end of 2015 (Boix, Miller, and Rosato, 2013, 2018) (years old)'][i]):
            democracy.append(df1['Entity'][i])
        else:
            other.append(df1['Entity'][i])
    except:
        other.append(df1['Entity'][i])

In [8]:
# Targets of non western countries.

def terror(df):
    known_df = df.loc[(df["gname"] != 'Unknown')]
    
    count = {}
    for i in known_df["gname"]:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1

    overig = 0
    pop_list = []
    for i, j in count.items():
        if j <= (len(known_df['gname'])/100)*2:
            pop_list.append(i)
            overig += j

    for key in pop_list:
        count.pop(key)

    count['Overig (onder 2%)'] = overig

    trace_1 = {
      "hole": 0.8,
      'text': [i for i, j in count.items()],
      "textposition":'outside', #yes
      "showlegend": False,
      "type": "pie",
      "values": [j for i, j in count.items()]
    }

    layout = {
      "title": "Aanslagen terroristische groepen (1970 - 2017) ", 
    }
    fig = go.Figure(data= [trace_1], layout=layout)
    iplot(fig)

In [9]:
def political_donut(year):
    if year == 1993:
        print('Terrorist database skipped this year :(')
        return False

    democracy = []
    other = []
    year_dif = 2015 - year

    for i in range(len(df1['Entity'])):
        try:
            if year_dif < int(df1['Age of democracies at the end of 2015 (Boix, Miller, and Rosato, 2013, 2018) (years old)'][i]):
                democracy.append(df1['Entity'][i])
            else:
                other.append(df1['Entity'][i])
        except:
            other.append(df1['Entity'][i])
    '''
    print('(democracy)', democracy)
    print('')
    print('(bad)', other)
    ''' 
    
    count = {}
    for i in df.loc[df['iyear'] == year]['country_txt']:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1

    dem = 0
    notdem = 0
    wut = 0
    check = []
    for i, j in count.items():
        if i in democracy:
            dem += j
        elif i in other:
            notdem += j
        else:
            check.append(i)
            wut += j
            
    count = {}
    count['Not democracy'] = notdem
    count['Democracy'] = dem
    count['Unknown'] = wut
    
    ###
    # % democracy vs not-democracy
    trace_1 = {
    "hole": 0.8,
    'text': ['Democracy', 'Not democracy'],  
    'marker': {'colors': ['rgb(0, 48, 240)', 'rgb(215, 11, 11)']},
    "textposition":'outside',
    "showlegend": False, 
    "type": "pie", 
    "values": [len(democracy), len(other)]
    }
    layout = {
        "title": 'Democracy vs not democracy (' + str(year) + ")", 
    }

    fig = go.Figure(data= [trace_1], layout=layout)
    
    iplot(fig)
    #Remove this in final version
    #print(count)
    
    trace_1 = {
      "hole": 0.8,
      'text': [i for i, j in count.items()],
      "textposition":'outside',
      "showlegend": False, 
      "type": "pie", 
      "values": [j for i, j in count.items()],
      'marker': {'colors': ['rgb(215, 11, 11)', 'rgb(0, 48, 240)', 'rgb(0, 0, 0)']}
    }

    layout = {
      "title": "Locaties aanslagen (" + str(year) + ")", 
    }
    fig = go.Figure(data= [trace_1], layout=layout)
    iplot(fig)
    
    #Remove this in final version
    #print('error', check)
political_slider = wg.IntSlider(
    value=2017,
    min=1970,
    max=2015,
    step=1,
    description='Year: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [10]:
def kill_confirmed(year):
    if year == 1993:
        print('Terrorist database skipped this year :(')
        return False

    democracy = []
    other = []
    year_dif = 2015 - year

    for i in range(len(df1['Entity'])):
        try:
            if year_dif < int(df1['Age of democracies at the end of 2015 (Boix, Miller, and Rosato, 2013, 2018) (years old)'][i]):
                democracy.append(df1['Entity'][i])
            else:
                other.append(df1['Entity'][i])
        except:
            other.append(df1['Entity'][i])
            
    countries = df['country_txt'].unique()
    killed = [0,0,0]
    wounded = [0,0,0]
    victims= [0,0,0]
    dc = df[df['iyear'] == year]

    for country in countries:
        cur_yr = dc[df['country_txt'] == country]
        wnd = cur_yr['nwound'].sum()
        kll = cur_yr['nkill'].sum()
        vctm = wnd + kll
        if country in democracy:
            victims[0] += vctm
            wounded[0] += wnd
            killed[0] += kll
        elif country in other:
            victims[1] += vctm
            wounded[1] += wnd
            killed[1] += kll
        
        else:
            victims[2] += vctm
            wounded[2] += wnd
            killed[2] += kll
    
    total = len(democracy) + len(other)
    countries = ['democracy ~' + str(round((len(democracy) /total)*100)) +'%', 'not democracy ~' + str(round((len(other) /total)*100)) +'%']
            
    sort_cty = [victims for _,victims in sorted(zip(victims, countries), reverse = True)]
    sort_vctm = sorted(victims, reverse = True)
    sort_kill = [victims for _,victims in sorted(zip(victims, killed), reverse = True)]
    sort_wnd = [victims for _,victims in sorted(zip(victims, wounded), reverse = True)]

    trace1 = go.Bar(
        x = sort_cty[:2],
        y = sort_kill[:2],
        name="Doden"
    )

    trace2 = go.Bar(
         x = sort_cty[:2],
         y = sort_wnd[:2],
         name="Gewonden"
     )

    plotlayout = go.Layout(
        title = go.layout.Title(
            text ='Doden en gewonden onder bestuursvormen (' + str(year) +')',
        ),
        xaxis=go.layout.XAxis(
            type = 'category',
            title = 'Bestuursvorm',
        ),
        yaxis = go.layout.YAxis(
            title = 'Slachtoffers'
        ),
        barmode='group'
    )

    fig = go.Figure(data=[trace1, trace2], layout=plotlayout)
    iplot(fig)
    print('De percentages geven aan hoeveel % van de wereld deze bestuursvorm heeft.')

kill_slider = wg.IntSlider(
    value=2017,
    min=1970,
    max=2015,
    step=1,
    description='Year: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [11]:
def count_democracies():
    
    countries = df['country_txt'].unique()
    year_of_dem = {}
    
    for country in countries:
        try:
            year_of_dem[country] = 2015 -  int(df1.loc[df1['Entity'] == country, 'Age of democracies at the end of 2015 (Boix, Miller, and Rosato, 2013, 2018) (years old)'].iloc[0])
        # if found years that country is a democracy is a string and can't be converted to int exception is thrown
        except:
            year_of_dem[country] = 2016
    
    dem_items = defaultdict(int)
    nondem_items = defaultdict(int)
    
    df2 = df.loc[:, ['iyear','country_txt', 'attacktype1_txt']]
    df2['casualties'] = df['nwound'] + df['nkill']
    
    for i, row in df2.iterrows():
        year, country, attacktype, casualties = list(row)
        if year_of_dem[country] < year:
            dem_items[attacktype] += casualties
        else:
            nondem_items[attacktype] += casualties
    
    return dem_items, nondem_items
            
dem_items, nondem_items = count_democracies()

## De invloed van politiek op terrorisme
Waar komt terrorisme vandaan en onder welke politieke omstandigheden is het het gevaarlijkst? Wonen in een democratie heeft het voordeel dat de burgers meer invloed uit kunnen oefenen op het bestuur van het land. In deze datastory zullen wij beargumenteren dat democratie ook terrorisme inperkt.

In [12]:
wg.interact(political_donut, year = political_slider);

interactive(children=(IntSlider(value=2015, continuous_update=False, description='Year: ', max=2015, min=1970)…

De bovenstaande donutcharts laten zien dat er meer aanslagen voorkomen in niet-democratische landen, terwijl er juist meer democratische landen in de wereld zijn. Verder terug in de tijd worden de meeste aanslagen gepleegt door democratische landen. De charts houden echter geen rekening met bevolkingsgrootte. Ook is te zien dat over tijd steeds meer landen democratisch worden.

In [13]:
wg.interact(kill_confirmed, year = kill_slider);

interactive(children=(IntSlider(value=2015, continuous_update=False, description='Year: ', max=2015, min=1970)…

Over de hele periode sinds 1970 is te zien dat zeker in de laatste jaren het gedeelte van de slachtoffers dat dodelijk is een stuk lager is dan in niet-democratische landen.

Uit de grafiek blijkt dat er veel meer doden en gewonden vallen in niet-democratische landen terwijl ze een minderheid opmaken t.o.v. bestuursvorm

In [14]:
sortedkeys = sorted(dem_items, key=str.lower)
nondem_casualties = []
dem_casualties = []

for key in sortedkeys:
    dem_casualties.append(dem_items[key])
    nondem_casualties.append(nondem_items[key])
    
trace_1 = go.Bar(
    x = sortedkeys,
    y = dem_casualties
)

trace_2 = go.Bar(
    x = sortedkeys,
    y = nondem_casualties
)

layout_1 = go.Layout(
    title = 'Democratisch'
)

layout_2 = go.Layout(
    title = 'Niet Democratisch'
)

#iplot(go.Figure(data=[trace_1], layout = layout_1))
#iplot(go.Figure(data=[trace_2], layout = layout_2))


Het is dus duidelijk dat er meer en dodelijkere aanslagen plaatsvinden in niet-democratische landen, maar hoe komt dat?
## Type aanslagen
Als we vergelijken welke soorten aanslagen het vaakst voorkomen in democratische en niet-democratische landen dan vallen een aantal interessante dingen op.
In niet-democratische landen is verreweg het grootste aandeel van terrorisme bomaanslagen 
In niet-democratische landen zijn gewapende en ongewapende aanvallen fysieke aanvallen veelvoorkomend en zijn er relatief veel vliegtuigovernames. In de treemaps hieronder zijn die verschillen te zien en in het staafdiagram daaronder zijn de verschillen uitgelicht.

In [15]:
color_brewer = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6']
categories = [k.replace(' ', '<br>') for k in sortedkeys]


def treemap(values, text, colours ,title):

    # dimensions treemap
    x = 0.
    y = 0.
    width = 100.
    height = 100.
    
    total_casualties = sum(values)
    percentages = [(v/total_casualties)*100 for v in values]
    
    normed = squarify.normalize_sizes(values, width, height)
    rects = squarify.squarify(normed, x, y, width, height)
    
    
    shapes = []
    annotations = []
    counter = 0

    for r in rects:
        shapes.append( 
            dict(
                type = 'rect', 
                x0 = r['x'], 
                y0 = r['y'], 
                x1 = r['x']+r['dx'], 
                y1 = r['y']+r['dy'],
                line = dict( width = .5 ),
                fillcolor = colours[counter]
            ) 
        )
        annotations.append(
            dict(
                x = r['x']+(r['dx']/2),
                y = r['y']+(r['dy']/2),
                text = '<b>' + text[counter] + '</b>',
                showarrow = False,
                font = dict(
                    size = (r['dx'] + r['dy'])/3,
                    family = 'Arial'
                )
            )
        )
        counter = counter + 1
        if counter >= len(color_brewer):
            counter = 0

    # For hover text
    trace = go.Scatter(
        x = [ r['x']+(r['dx']/2) for r in rects ], 
        y = [ r['y']+(r['dy']/2) for r in rects ],
        text = [ f"{int(values[i])} casualties <br>{percentages[i]:.1f} % of total" for i in range(len(values))],
        hoverinfo = 'text'
    )

    layout = dict(
        title = title,
        height=1000, 
        width=1000,
        xaxis=dict(showgrid=False,zeroline=False),
        yaxis=dict(showgrid=False,zeroline=False),
        shapes=shapes,
        annotations=annotations,
        hovermode='closest'
    )
    
    return trace, layout, percentages

trace0, layout0, percentages0 = treemap(dem_casualties, categories, color_brewer, 'Types of terrorist attacks in democratic countries')
trace1, layout1, percentages1 = treemap(nondem_casualties, categories, color_brewer, 'Types of terrorist attacks in non-democratic countries')



# Without hovertext
# figure = dict(data=[Scatter()], layout=layout)

iplot(dict(data=[trace0], layout=layout0), filename='treemap_democratic')
iplot(dict(data=[trace1], layout=layout1), filename='treemap_democratic')

In [16]:
import numpy as np

democracy_colour = 'rgb(215, 11, 11)'
nondemocracy_colour = 'rgb(0, 48, 240)'

percentages_diff = list(np.array(percentages1)-np.array(percentages0))
rev_keys = sortedkeys[::-1]

rev_percentages_diff = percentages_diff[::-1]

colours_differences = [democracy_colour if x<0 else nondemocracy_colour for x in rev_percentages_diff]


trace_1 = go.Bar(
    name = 'Non Democratic countries',
    y = rev_keys,
    x = rev_percentages_diff,
    orientation = 'h',
    marker = dict(color = colours_differences)
)

layout = go.Layout(
    title = 'Differences',
    legend = dict(x = 0.7, y = 0.2),
    width = 1000,
    showlegend = True,
    xaxis = dict(
        zeroline=True,
        showline=False,
        showticklabels=True,
        showgrid=True,
        side='top',
        dtick=1,
        ticksuffix = ".0%"
    ),
    margin=go.layout.Margin(
        l=275,
        r=50,
        b=50,
        t=150
    ),
    annotations = [dict(
            x=1,
            y=1.15,
            showarrow=False,
            text='Percentueel verschil (%)',
            xref='paper',
            yref='paper'
        )],
)

trace_2 = go.Bar(
    name = 'Democratic countries',
    x = [None],
    y = [None],
    marker = dict(color = democracy_colour),
    showlegend = True
)
iplot(go.Figure(data=[trace_1, trace_2], layout=layout))


In [17]:
terror(df)

## Politieke stabiliteit
In de kaart hieronder is te zien dat de politieke stabiliteit in democratische landen hoger is. Deze index is gebaseerd op de controle van corruptie en effectiviteit van de overheid op het gebied van wetgeving en uitvoering. Het is niet vanzelfsprekend dat democratie op zich de reden is dat deze landen minder terrorisme hebben, maar het zou kunne dat de effectiviteit van de overheid ervoor zorgt dat overheden in democratische landen meer grip hebben op ernstige terroristische aanslagen zoals bomaanslagen en er daarom uiteindelijk minder mensen omkomen.

In [18]:
wg.interact(Scatterplot, year = year)

interactive(children=(IntSlider(value=2017, continuous_update=False, description='Year: ', max=2017, min=1996)…

<function __main__.Scatterplot(year)>

In [19]:
df_wgi
df_wgi.columns = df_wgi.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(','').str.replace(')','')

In [20]:
democratic_countries = pd.DataFrame()
otherdf = pd.DataFrame()
cyear = str(2015) + "_[yr" + str(2015) + ']'
cyear2= df_wgi.loc[:, ['country_name', 'series_code', cyear]]
tyear = df.loc[df['iyear'] == 2015]
countries = tyear['country_txt'].unique()

for country in countries:
    if country in democracy:
        cyear3 = cyear2.loc[cyear2['series_code'] == 'PV.EST']
        cyear4 = cyear3.loc[cyear3['country_name']== country]
        democratic_countries = democratic_countries.append(cyear4, ignore_index=True)
    elif country in other:
        cyear5 = cyear2.loc[cyear2['series_code'] == 'PV.EST']
        cyear6 = cyear5.loc[cyear5['country_name']== country]
        otherdf = otherdf.append(cyear6, ignore_index=True)


In [21]:
data1 = [go.Choropleth(
    colorscale = 'Viridis',
    z = democratic_countries['2015_[yr2015]'],
    locations = democratic_countries['country_name'],
    locationmode = 'country names',
    colorbar = go.choropleth.ColorBar(
        title = "Political Stability")
),
    go.Choropleth(
    colorscale = 'Viridis',
    z = otherdf['2015_[yr2015]'],
    locations = otherdf['country_name'],
    locationmode = 'country names',
    colorbar = go.choropleth.ColorBar(
        title = "Political Stability")
)
]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Politieke Stabiliteit van Democratische Landen'
    ),
    updatemenus = list([
      dict(
        #y = 0.8,
        buttons = list([
            dict(label='Democratic Countries',
                method = 'update',
                args = [{'visible': [True, False]}]),
            dict(label='Non-democratic Countries',
                method = 'update',
                args = [{'visible': [False, True]}])
        ])
      )
    ]),
    geo = go.layout.Geo(
        scope = 'world',
        projection = go.layout.geo.Projection(type = 'equirectangular'),
        showcountries = True),

)

fig = go.Figure(data = data1, layout = layout)
iplot(fig)